*Load packages and data:*

In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'LoadVariables', 'TransmissionLines', 'GeneratorsVariables'],
        'maps': ['LoadMaps','GeneratorsMaps'],
        'variable2D': ['HourlyVariation'],
        'scalars': ['Scalars']}
db = dbFromWB(os.path.join(d['data'],'E4.xlsx'), read)
readSets(db)

# Exercise 4: Technology Complementarity

The main objective of the fourth exercise is to estimate how technologies may complement each other. Specifically, we will look at a number of scenarios:
* *High/low shares of renewables*.
* *High/low shares of CHP plants*.
* *Large/small heat markets*.

The exercise builds on the ```mBasicPH``` model class; you can find helpful information on this in the following:
* The note "*Models in Energy Economics*" describes the model in section 4.
* The notebook [M_mBasicPH](M_mBasicPH.ipynb) includes some description of the model class.
* The notebook [E4_SolutionGuide](E4_SolutionGuide.ipynb) provides a short solution to the problems posted in the current notebook.

*Note: The model class ```mBasicPH``` automatically includes transmission lines and geographic markets. Currently, the class needs to have multiple geographic areas. We have set the transmission capacity between $g_1,g_2$ to zero, and only focus on $g_1$ - so, we ignore the effect of trade throughout. Technically, though, this means that we have to keep extracting the right information from $g_1$. The following should help subsetting the relevant symbols throughout:*

In [2]:
ids = db['id'][db['id'].str.startswith('g1')]
g   = pd.Index(['g1'],name='g')
hvts= db['hvt'][db['hvt'].str.startswith('g1')]
cE  = pd.Index(['c1'],name='c_E')
cH  = pd.Index(['c2'],name='c_H')

## The setup:

We consider a ```mBasicPH``` model with $H=4$ hours. The hourly average load is $100 GJ$ for both power and heat markets.

#### Hourly Variation;

We use a stylized hourly variation pattern where the four hours represent four 6-hour blocks of the day (hours 0-6, 6-12, 12-18, 18-24). Heat demand is slightly higher in the night ($h=1,4$), whereas electricity is somewhat higher during the day $(h=2,3$). Wind generation is slightly higher at night, whereas photovoltaics and solar heating primarily produce during the day.

In [3]:
DFVariation = rc_pd(db['CapVariation'], hvts).unstack('hvt').assign(LoadVariation_E = db['LoadVariation_E'].xs('c1'), LoadVariation_H = db['LoadVariation_H'].xs('c1'))
DFVariation

hvt,g1_WS,g1_PV,g1_SH,LoadVariation_E,LoadVariation_H
h,,,,,
1,0.5,0,0.15,0.1,0.35
2,0.3,0.3,0.25,0.3,0.15
3,0.6,0.4,0.25,0.5,0.15
4,0.7,0.05,0.15,0.1,0.35


With only four hours, this stylized setup produces some pretty correlated variables:

In [4]:
DFVariation.astype(float).corr().round(2)

hvt,g1_WS,g1_PV,g1_SH,LoadVariation_E,LoadVariation_H
hvt,,,,,
g1_WS,1.00,-0.29,-0.51,-0.15,0.51
g1_PV,-0.29,1.00,0.97,0.97,-0.97
g1_SH,-0.51,0.97,1.00,0.90,-1.00
LoadVariation_E,-0.15,0.97,0.90,1.00,-0.90
LoadVariation_H,0.51,-0.97,-1.00,-0.90,1.00


#### Technologies:

We include the following technologies:
* Condensation plants (```CD```): Simple electricity producing, dispatchable plants. Three versions: One that relies on coal, one on natural gas, and one that relies on biomass.
* Boiler Heater plants (```BH```): Simple heat producing, dispatchable plants. Two versions: One that relies on natural gas, one that relies on biomass.
* Back-pressure plants (```BP```): Combined heat and power plants. Two versions: One relies on coal, another on natural gas.
* Wind and solar plants (```WS,PV,SH```): Intermittent plants. WS,PV produce electricity. SH produce heat. 
* Heat pump (```HP```): Relies on electricity to produce heat. 

### Problem 1: Baseline scenario

*Solve the model in the baseline scenario (data as is) and comment.*

### Problem 2: Suggest CHP plant strategy

*Based on your results in problem 1, suggest a better technology mix that includes CHP plants. Keep the total generating capacity for electricity and heat roughly constant to isolate the effect of changing the composition of plants. Simulate and comment on your result.*

### Problem 3: Marginal system costs with large shares of CHP

*Update generating capacities to the ones detailed in the sheet ```UpdateCaps``` as follows:*

In [5]:
newCaps = dbFromWB(os.path.join(d['data'],'E4.xlsx'), {'variables': ['UpdateCaps']})
db.symbols.update(**newCaps.symbols)

*Note that the sum of capacities has increased. Also, instead of only relying on simple condensation and boiler heating plants, we now have all types of plants. Re-solve the model and comment on the solution. Specifically, answer why electricity prices are high in $h=2,3$ and heat prices in $h=1,4$.*

### Problem 4: Renewables and heat pumps.

*Vary the generating capacity of wind and solar from 50 to 200 GJ. For each step in this loop, vary the installed capacity of heat pumps from 25-100 as well.*

*Hint: You can use the following syntax to perform create 1d loop that can be used with the ```loopSolveExtract``` method used in the previous exercises*

```Python
# Create two separate grids for Cap_E and Cap_H:
loop1 = pd.Index(range(11), name = 'l1')
loop2 = pd.Index(range(11), name = 'l2')
v0_ECap = pd.Series(50, index =  mBasicPH.getTechs_i(['PV','WS'],db))
vT_ECap = pd.Series(200, index = v0_ECap.index)
v0_HCap = pd.Series(25, index = mBasicPH.getTechs_i('HP',db))
vT_HCap = pd.Series(100, index = mBasicPH.getTechs_i('HP',db))
grid_ECap = addGrid(v0_ECap,vT_ECap,loop1,'GeneratingCap_E')
grid_HCap = addGrid(v0_HCap,vT_HCap,loop2,'GeneratingCap_H')
# Collapse the two grids into two 1d index:
df_E = lpCompiler.sortAll(addGrid(grid_ECap, grid_ECap, loop2, 'GeneratingCap_E').unstack('id'))
df_H = lpCompiler.sortAll(addGrid(grid_HCap, grid_HCap, loop1, 'GeneratingCap_H').unstack('id'))
df_E.index = df_E.index.to_flat_index().rename('loop')
df_H.index = df_H.index.to_flat_index().rename('loop')
grid_ECap_1d = df_E.stack().rename('GeneratingCap_E')
grid_HCap_1d = df_H.stack().rename('GeneratingCap_H')
grids = [grid_ECap_1d, grid_HCap_1d]
loop = df_E.index
```